# Logistic Regression Example Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import *
from pyspark.sql import SQLContext, Row
sqlContext = SQLContext(sc)

In [2]:
data = sqlContext.read.format("com.databricks.spark.csv").options(header='true').options(inferSchema='true').load("file:/customer_churn.csv")

In [3]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [4]:
data.describe().show()

+-------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|summary|              Age|    Total_Purchase|   Account_Manager|             Years|         Num_Sites|              Churn|
+-------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|  count|              900|               900|               900|               900|               900|                900|
|   mean|41.81666666666667|10062.824033333332|0.4811111111111111|5.2731555555555545| 8.587777777777777|0.16666666666666666|
| stddev|6.127560416916252|2408.6445318580945|0.4999208935073339|1.2744490131946158|1.7648355920350964|0.37288521227723564|
|    min|             22.0|             100.0|                 0|               1.0|               3.0|                  0|
|    max|             65.0|          18026.01|                 1|              9.15|              14.0|                  1|
+-------

In [5]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [6]:
from pyspark.sql.types import DoubleType
my_final_data = data.withColumn("Age", data.Age.cast(DoubleType()))
my_data = my_final_data.withColumn("Churn", data.Churn.cast(DoubleType()))
my_data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: double (nullable = true)



In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [9]:
output = assembler.transform(my_data)

In [10]:
final_data = output.select('features','Churn')

In [11]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

In [12]:
from pyspark.ml.classification import LogisticRegression

In [13]:
lr_churn = LogisticRegression(labelCol='Churn')
train_churn.show()

+--------------------+-----+
|            features|Churn|
+--------------------+-----+
|[26.0,8787.39,1.0...|  1.0|
|[29.0,11274.46,1....|  0.0|
|[30.0,7960.64,1.0...|  1.0|
|[30.0,8403.78,1.0...|  0.0|
|[30.0,8677.28,1.0...|  0.0|
|[30.0,10744.14,1....|  1.0|
|[30.0,10960.52,1....|  0.0|
|[30.0,11575.37,1....|  1.0|
|[30.0,12788.37,0....|  0.0|
|[31.0,11297.57,1....|  1.0|
|[31.0,11743.24,0....|  0.0|
|[32.0,7896.65,0.0...|  0.0|
|[32.0,8617.98,1.0...|  1.0|
|[32.0,9472.72,1.0...|  0.0|
|[32.0,9885.12,1.0...|  1.0|
|[32.0,12254.75,1....|  0.0|
|[32.0,12479.72,0....|  0.0|
|[32.0,12547.91,0....|  0.0|
|[33.0,7492.9,0.0,...|  0.0|
|[33.0,7750.54,1.0...|  0.0|
+--------------------+-----+
only showing top 20 rows



In [20]:
fitted_churn_model = lr_churn.fit(train_churn)

In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [22]:
pred_and_labels = fitted_churn_model.transform(test_churn)

/usr/lib/spark/python/pyspark/ml/classification.py:207: UserWarning: weights is deprecated. Use coefficients instead.
  warnings.warn("weights is deprecated. Use coefficients instead.")


In [23]:
final_lr_model = lr_churn.fit(final_data)

In [24]:
new_customers = sqlContext.read.format("com.databricks.spark.csv").options(header='true').options(inferSchema='true').load("file:/new_customers.csv")

In [25]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [26]:
test_new_customers = assembler.transform(new_customers)

In [27]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [28]:
final_results = final_lr_model.transform(test_new_customers)

In [29]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       0.0|
+----------------+----------+



## Great Job!